Run the following line if you don't have any of the libraries

In [ ]:
!pip install statsmodels pandas numpy --upgrade --user

In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.api import acf, pacf, graphics
import data_prep_modified as dpm
import numpy as np

In [2]:
data = pd.read_csv("./data/output.csv")
list(data.columns) 

['Date',
 'Open_Pharm',
 'High_Pharm',
 'Low_Pharm',
 'Close_Pharm',
 'Adj Close_Pharm',
 'Volume_Pharm',
 'Open_Semicon',
 'High_Semicon',
 'Low_Semicon',
 'Close_Semicon',
 'Adj Close_Semicon',
 'Volume_Semicon',
 'Open_SNP500',
 'High_SNP500',
 'Low_SNP500',
 'Close_SNP500',
 'Adj Close_SNP500',
 'Volume_SNP500',
 'Open_Industrial',
 'High_Industrial',
 'Low_Industrial',
 'Close_Industrial',
 'Adj Close_Industrial',
 'Volume_Industrial',
 'Open_Energy',
 'High_Energy',
 'Low_Energy',
 'Close_Energy',
 'Adj Close_Energy',
 'Volume_Energy',
 'Open_Financial',
 'High_Financial',
 'Low_Financial',
 'Close_Financial',
 'Adj Close_Financial',
 'Volume_Financial',
 'Open_Tech',
 'High_Tech',
 'Low_Tech',
 'Close_Tech',
 'Adj Close_Tech',
 'Volume_Tech',
 'Open_Utilities',
 'High_Utilities',
 'Low_Utilities',
 'Close_Utilities',
 'Adj Close_Utilities',
 'Volume_Utilities',
 'Open_Consumer',
 'High_Consumer',
 'Low_Consumer',
 'Close_Consumer',
 'Adj Close_Consumer',
 'Volume_Consumer']

In [4]:
#variable prep commands, replace snp500 with variable of choice
snp500delta = dpm.get_perc_return(data, column_name="Adj Close_SNP500")
snp500delta.set_index('Date', inplace=True)
#for some reason, I cant put the set_index command in the function because of some error

In [5]:
snp500delta.head()

,%_returns
Date,
5/1/2010,0.003116
6/1/2010,0.000546
7/1/2010,0.004001
8/1/2010,0.002882
11/1/2010,0.001747


In [6]:
ar_1 = AutoReg(snp500delta, lags =1).fit()
ar_1.summary()

C:\Users\Hamu\AppData\Roaming\Python\Python37\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\Hamu\AppData\Roaming\Python\Python37\site-packages\statsmodels\tsa\base\tsa_model.py:222: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  ' forecasting.', ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:              %_returns   No. Observations:                 2570
Model:                     AutoReg(1)   Log Likelihood                8102.487
Method:               Conditional MLE   S.D. of innovations              0.010
Date:                Mon, 06 Apr 2020   AIC                             -9.143
Time:                        14:01:12   BIC                             -9.137
Sample:                             1   HQIC                            -9.141
                                 2570                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
intercept        0.0004      0.000      1.882      0.060   -1.59e-05       0.001
%_returns.L1    -0.1533      0.020     -7.836      0.000      -0.192      -0.115
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1           -6.5230           +0.0000j            6.5230            0.5000
-----------------------------------------------------------------------------
"""